<a href="https://colab.research.google.com/github/ydaigo/The-4th-Tellus-Satellite-Challenge/blob/main/satelite_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 準備
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==1.15
!pip uninstall keras -y
!pip install keras==2.2.4
import os
import sys
import random
import warnings
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import tifffile
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 接続切れ対策
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

Uninstalling tensorflow-2.3.0:


In [ ]:
#@title 前処理
im_width = 512
im_height = 512
im_chan = 1
train_ids = [1,2,3,4,5,6,7,11,13,14,15,16,17,18,19,21,22,23,24]
test_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
            ,20,21,22,23,24,25,26,27,28,29]
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()

sizes_train = []
for n,id_ in enumerate(train_ids):
    img = load_img(f'/content/drive/My Drive/signate/satelite/train_images_png_full/train_gray_{id_:02d}.png')
    x = img_to_array(img)[:,:,1]
    sizes_train.append([x.shape[0], x.shape[1]])
    x = resize(x, (im_height, im_width, 1), mode='constant', preserve_range=True)
    X_train[n] = x
    mask = img_to_array(load_img(f'/content/drive/My Drive/signate/satelite/mask_full_v2/train_mask_{id_:02d}.png'))[:,:,1]
    mask = np.where(mask==255,0,255)
    Y_train[n] = resize(mask, (im_width, im_height, 1), mode='constant', preserve_range=True)
sizes_test = []
for n, id_ in enumerate(test_ids):
    img = load_img(f'/content/drive/My Drive/signate/satelite/test_images_png_full/test_gray_{id_:02d}.png')
    x = img_to_array(img)[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    x = resize(x, (im_height, im_width, 1), mode='constant', preserve_range=True)
    X_test[n] = x

print('Done!')

In [ ]:
# generator
data_gen_args  =dict(featurewise_center=False, 
                             samplewise_center=False, 
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False, 
                             zca_whitening=False, 
                             zca_epsilon=1e-06, 
                             rotation_range=90, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             brightness_range=None,
                             shear_range=90, zoom_range=0.0,
                             channel_shift_range=0.0,
                             fill_mode='constant', cval=0.0,
                             horizontal_flip=True, 
                             vertical_flip=True, 
                             rescale=None, 
                             preprocessing_function=None, 
                             data_format=None,
                             validation_split=0.0)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
seed = 1
image_datagen.fit(X_train[:17], augment=True, seed=seed)
mask_datagen.fit(Y_train[:17], augment=True, seed=seed)
image_generator = image_datagen.flow(X_train[:17], y=None, batch_size=16, shuffle=False, seed=0, save_to_dir=None, save_prefix='', save_format='png', subset=None)
mask_generator = mask_datagen.flow(Y_train[:17], y=None, batch_size=16, shuffle=False, seed=0, save_to_dir=None, save_prefix='', save_format='png', subset=None)
train_generator = zip(image_generator, mask_generator)


In [ ]:
#@title モデル
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)
# Build U-Net model
inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

In [ ]:
#title 学習
import keras
model = load_model('/content/drive/My Drive/signate/satelite/weight/checkpoint/30.h5', custom_objects={'mean_iou': mean_iou})
class nvidia(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        !nvidia-smi
earlystopper = EarlyStopping(patience=500, verbose=1)
nvidiaCallback = nvidia()
checkpointer = ModelCheckpoint('/content/drive/My Drive/signate/satelite/weight/checkpoint/model-tgs-salt-1.h5', verbose=1, save_best_only=True,monitor='val_mean_iou', mode='max')
results = model.fit_generator(train_generator, validation_data=(X_train[17:],Y_train[17:]), steps_per_epoch=300, epochs=300,
                    callbacks=[earlystopper, checkpointer],shuffle=False)

In [ ]:
#@title 予測
model = load_model('/content/drive/My Drive/signate/satelite/weight/checkpoint/model-tgs-salt-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
#@title 可視化
for i in range(len(preds_test_t)):
  plt.imshow(np.squeeze(X_test[i]))
  plt.colorbar()
  plt.show()
  plt.imshow(np.squeeze(preds_test_t[i]))
  plt.show()
  img = np.squeeze(preds_test_t[i]).astype(np.float64)
  img = np.round(cv2.resize(img,(sizes_test[i][1],sizes_test[i][0])))
  # u, counts = np.unique(img, return_counts=True)
  # print(u)
  # # [ 0 10 20 30]

  # print(counts)
  #img = resize(img, (sizes_test[0][0], sizes_test[0][1]), mode='constant', preserve_range=True)
  #contours,hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  kernel_x = np.array([[0,0,0],
                    [0,1,-1],
                    [0,0,0]])
  kernel_y = np.array([[0,0,0],
                    [0,1,0],
                    [0,-1,0]])
  img = np.where(cv2.filter2D(img, -1, kernel_x)!=0,1,0)+np.where(cv2.filter2D(img, -1, kernel_y)!=0,1,0)
  plt.imshow(img)

  plt.show()
  print(len(list(zip(*np.where(img!=0)))))

In [ ]:
#@title 後処理 ファイルの作成
json_ ={}
for id_,img in enumerate(preds_test_t):
  img =np.squeeze(img).astype(np.float64)
  img = np.round(cv2.resize(img,(sizes_test[id_][1],sizes_test[id_][0]),interpolation=cv2.INTER_NEAREST))
  kernel_x = np.array([[0,0,0],
                    [0,1,-1],
                    [0,0,0]])
  kernel_y = np.array([[0,0,0],
                    [0,1,0],
                    [0,-1,0]])
  img = np.where(cv2.filter2D(img, -1, kernel_x)!=0,1,0)+np.where(cv2.filter2D(img, -1, kernel_y)!=0,1,0)
  #print(sizes_test[id_])
  #print(np.unique(img))
  #border = list(zip(*np.where(img!=0)))
  print(len(list(zip(*np.where(img>0)))))
  if len(list(zip(*np.where(img>0))))<29000:
    json_[f'test_{id_:02d}.tif'] = [[int(j),int(i)] for i,j in zip(*np.where((img!=0)))]
  else:
    json_[f'test_{id_:02d}.tif'] = [[0,0]]
with open("/content/submit.json", "w") as f:
    json.dump(json_, f)

In [ ]:
json_[f'test_26.tif']

In [ ]:
json_ ={}
for id_,img in enumerate(Y_train[:1]):
  img =np.squeeze(img).astype(np.float64)
  img = cv2.resize(img,(sizes_train[id_][1],sizes_train[id_][0]),interpolation=cv2.INTER_NEAREST)
  print(np.unique(img))
  kernel_x = np.array([[0,0,0],
                    [0,1,-1],
                    [0,0,0]])
  kernel_y = np.array([[0,0,0],
                    [0,1,0],
                    [0,-1,0]])
  img = np.where(cv2.filter2D(img, -1, kernel_x)!=0,1,0)+np.where(cv2.filter2D(img, -1, kernel_y)!=0,1,0)
  #print(sizes_test[id_])
  print(np.unique(img))
  #border = list(zip(*np.where(img!=0)))
  print(len(list(zip(*np.where(img>0)))))
  id_ = 2
  json_[f'train_{id_:02d}.tif'] = [[int(j),int(i)] for i,j in zip(*np.where((img!=0)))]
with open("/content/train.json", "w") as f:
    json.dump(json_, f)

In [ ]:
img =np.squeeze(Y_train[0]).astype(np.float64)
img = cv2.resize(img,(sizes_train[id_][1],sizes_train[id_][0]),interpolation=cv2.INTER_NEAREST)
kernel_x = np.array([[0,0,0],
                  [0,1,-1],
                  [0,0,0]])
kernel_y = np.array([[0,0,0],
                  [0,1,0],
                  [0,-1,0]])
img = np.where(cv2.filter2D(img, -1, kernel_x)!=0,1,0)+np.where(cv2.filter2D(img, -1, kernel_y)!=0,1,0)
plt.imshow(np.squeeze(img))
np.unique(img)
cv2.imwrite('a.png', img)

In [ ]:
img = np.array([[0,0,0,0,0],
                [0,0,0,0,0],
                [1,1,1,1,1],
                [0,0,0,0,0],
                [0,0,0,0,0]]).astype(np.float32)
img = cv2.resize(img,(100,100),interpolation  = cv2.INTER_LINEAR)
plt.imshow(img)

In [ ]:
Y_train[0].dtype

In [ ]:
#　評価
img = img_to_array(load_img(f'/content/train_mask_02.png'))[:,:,1]
img = cv2.resize(img,(128,128),interpolation=cv2.INTER_NEAREST)
img = cv2.resize(img,(sizes_train[0][1],sizes_train[0][0]),interpolation=cv2.INTER_NEAREST)
print(np.unique(img))
kernel_x = np.array([[0,0,0],
                  [0,1,-1],
                  [0,0,0]])
kernel_y = np.array([[0,0,0],
                  [0,1,0],
                  [0,-1,0]])
img = np.where(cv2.filter2D(img, -1, kernel_x)!=0,1,0)+np.where(cv2.filter2D(img, -1, kernel_y)!=0,1,0)
id_ = 2
json_[f'train_{id_:02d}.tif'] = [[int(j),int(i)] for i,j in zip(*np.where((img!=0)))]
with open("/content/train.json", "w") as f:
    json.dump(json_, f)
with open('/content/drive/My Drive/signate/satelite/train_annotations/train_02.json') as f:
    df = json.load(f)
aa = {}
aa["train_02.tif"] = df
with open("/content/ano.json", "w") as f:
    json.dump(aa, f)
!python /content/drive/My\ Drive/signate/satelite/evaluate.py \
  --prediction-file /content/train.json \
  --annotation-file /content/ano.json

In [ ]:
mask = img_to_array(load_img(f'/content/train_mask_02.png'))[:,:,1]
mask.shape

In [ ]:
json_